In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.patches as patches

EXPORT_DIR = Path("../reports/exports"); EXPORT_DIR.mkdir(parents=True, exist_ok=True)

plt.rcParams.update({
    "figure.figsize": (10, 4.8),
    "axes.titlesize": 16,
    "axes.labelsize": 13,
    "xtick.labelsize": 11,
    "ytick.labelsize": 11,
})


In [3]:
def draw_box(ax, xy, w, h, text, fc="#EFF3FF", ec="#1F4B99", lw=1.8, r=0.15, txtsize=11, bold=False):
    box = patches.FancyBboxPatch(
        xy, w, h,
        boxstyle=patches.BoxStyle("Round", rounding_size=r*min(w,h)),
        linewidth=lw, edgecolor=ec, facecolor=fc
    )
    ax.add_patch(box)
    ax.text(xy[0]+w/2, xy[1]+h/2, text, ha="center", va="center",
            fontsize=txtsize, fontweight=("bold" if bold else "normal"), wrap=True)

def arrow(ax, xy1, xy2):
    ax.annotate("", xy=xy2, xytext=xy1,
                arrowprops=dict(arrowstyle="->", lw=1.8, color="#333"))

fig, ax = plt.subplots(figsize=(12,4.8))
ax.axis("off")

# layout grid (x in [0,1], y in [0,1])
boxes = [
    ((0.02, 0.35), 0.16, 0.30, "IllustrisTNG100\nSnapshots 18–33\nMPB Extraction\n(≥90% coverage, N≈2500)", True),
    ((0.22, 0.35), 0.16, 0.30, "Preprocessing\n• Select features\n• Handle coverage\n• Standardize (z-score)", False),
    ((0.42, 0.35), 0.16, 0.30, "Train/Val/Test Split\nby Subhalo ID", False),
    ((0.62, 0.35), 0.16, 0.30, "Modeling\nLSTM Forecaster\n(H=[1,3,5])", False),
    ((0.82, 0.35), 0.16, 0.30, "Evaluation\n• RMSE per horizon/feature\n• Parity plots\n• Baselines\n• Explainability", False),
]

for (xy, w, h, txt, bold) in boxes:
    draw_box(ax, xy, w, h, txt, bold=bold)

arrow(ax, (0.18, 0.50), (0.22, 0.50))
arrow(ax, (0.38, 0.50), (0.42, 0.50))
arrow(ax, (0.58, 0.50), (0.62, 0.50))
arrow(ax, (0.78, 0.50), (0.82, 0.50))

ax.set_title("End-to-End Pipeline for Forecasting Black Hole Evolution")
plt.tight_layout()
fig.savefig(EXPORT_DIR/"pipeline_schematic.png", dpi=300, bbox_inches="tight")
fig.savefig(EXPORT_DIR/"pipeline_schematic.svg", bbox_inches="tight")
plt.close(fig)

str(EXPORT_DIR/"pipeline_schematic.png")


'../reports/exports/pipeline_schematic.png'

In [4]:
fig, ax = plt.subplots(figsize=(12,4.8))
ax.axis("off")

# input feature labels
in_feats = ["BH Mass", "BH Accretion", "Stellar Mass", "SFR", "Halo Mass", "Velocity Dispersion"]
in_text = "Input at each timestep (z-scored):\n" + ", ".join(in_feats)

# draw time steps t-(T-1) ... t
T = 8
x0, y0 = 0.06, 0.25
dx = 0.09
h = 0.20; w = 0.08

# input row
for i in range(T):
    draw_box(ax, (x0 + i*dx, y0), w, h, f"x[{i+1}]", fc="#F2F2F2", ec="#666", txtsize=10)

ax.text(x0 + (T*dx)/2, y0-0.12, in_text, ha="center", va="center", fontsize=11)

# LSTM cells row
y1 = y0 + 0.28
for i in range(T):
    draw_box(ax, (x0 + i*dx, y1), w, h, "LSTM", fc="#E8F5E9", ec="#1B5E20", txtsize=11, bold=True)

# recurrent arrows
for i in range(T-1):
    arrow(ax, (x0 + i*dx + w, y1 + h/2), (x0 + (i+1)*dx, y1 + h/2))

# input→cell arrows
for i in range(T):
    arrow(ax, (x0 + i*dx + w/2, y0 + h), (x0 + i*dx + w/2, y1))

# output head
x_head = x0 + (T-1)*dx + 0.12
draw_box(ax, (x_head, y1), 0.14, h, "Dense Head", fc="#FFF3E0", ec="#E65100", txtsize=11, bold=True)
arrow(ax, (x0 + (T-1)*dx + w, y1 + h/2), (x_head, y1 + h/2))

# multi-horizon outputs
y_outs = [y1 + 0.25, y1 + 0.10, y1 - 0.05]
for k, (hoff, label) in enumerate(zip([1,3,5], ["H=1", "H=3", "H=5"])):
    draw_box(ax, (x_head + 0.18, y_outs[k]-0.06), 0.16, 0.12, f"Predicted features\n({label})", fc="#E3F2FD", ec="#0D47A1", txtsize=10)
    arrow(ax, (x_head + 0.14, y1 + h/2), (x_head + 0.18, y_outs[k]))

ax.set_title("LSTM Architecture for Multi-Horizon Forecasting")
plt.tight_layout()
fig.savefig(EXPORT_DIR/"lstm_architecture.png", dpi=300, bbox_inches="tight")
fig.savefig(EXPORT_DIR/"lstm_architecture.svg", bbox_inches="tight")
plt.close(fig)

[str(EXPORT_DIR/"lstm_architecture.png"), str(EXPORT_DIR/"lstm_architecture.svg")]


['../reports/exports/lstm_architecture.png',
 '../reports/exports/lstm_architecture.svg']